In [1]:
import re
import nltk


from collections import Counter, OrderedDict
from sklearn.base import TransformerMixin,BaseEstimator
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.util import ngrams
import numpy as np
import hashlib
import math
import pandas as pd 
import matplotlib.pyplot as plt
import random
import string
from nltk.tokenize import WordPunctTokenizer

from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, precision_score, recall_score,classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC
import spacy
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import preprocessing

from bs4 import BeautifulSoup
from spacy.lang.en import English

from pylab import barh,plot,yticks,show,grid,xlabel,figure,cla,close
from sklearn.linear_model import LinearRegression

# Regex

. - matches any character, except newline.

\d, \s \S - match digit, match whitespace, not whitespace.

\b, \B - word, not word boundary.

[xyz] - matches x, y or z.

[^xyz] - matches anything that is not x, y or z.

[x-z] - matches a character between x and z.

^xyz$ - ^ is the start of the string, $ is the end of the string.

\. - use escaping to match special characters.

\t, \n - matches tab and newline.

x* - matches 0 or more symbols x.

x+ - matches 1 or more symbols x.

x? - matches 0 or 1 symbol x.

.?, *?, +?, etc - represent non-greedy search.

x{5} - matches exactly 5 symbols x.

x{5,} - matches 5 or more symbols x.

x{5, 8} - matches between 5 and 8 symbols x.

xy|yz - matches xy or yz.

In [3]:
#nltk.download('stopwords')
#stemmer = SnowballStemmer("english", ignore_stopwords=True)
#stems = [list(map(stemmer.stem, words))]

# Read the data

In [ ]:
#Load the dataset
##there are two columns in the data set sentiment which is positive and negative and text columns this is movie review
#df = pd.read_csv('./datasets/twitter.csv')
df = pd.read_csv('./data/covid19_data.csv', encoding='latin1')
#df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1,inplace=True)
df = df[['class','title']]
df.rename(columns={"class":"target", "title":"text"},inplace=True)

df.head()
#NOTE: If the data set is so many choose part of that in order to accelerate the process of analysing
    ##docs = df.text[:200]
# Get the text
#docs = df['text']



###we can read from file and represent it as list as follows
#def file_to_list(file_name):
    #with open(file_name, 'r', encoding='utf-8') as f:
        #return [line.strip() for line in f.readlines()]
    
#X_train_pre = file_to_list('data/tweets_train_preprocessed.txt')
#X_dev_pre = file_to_list('data/tweets_dev_preprocessed.txt')
#X_test_pre = file_to_list('data/tweets_test_preprocessed.txt')

##############################################################Read a JSON file################################

#docs = []
#with open('./datasets/sample_data.json') as fp:
    #for line in fp:
        #entry = json.loads(line)
        #docs.append(entry['body'])
        
#print('I read {} documents'.format(len(docs)))

In [ ]:
df['target'].unique()

In [ ]:
#drop some columns
dr = ['Unverified','Mixed','Collections','Fake news','MiSLEADING','IN DISPUTE', 'Misinformation / Conspiracy theory','PANTS ON FIRE'
   ,'Suspicions','NO EVIDENCE',]
df.drop(df['class'] is in [], axis=1,inplace=True)

Define a function to clean target and transform it to false and true

In [ ]:
def Clean_col_target(df_):
    for targ in df_['target'].values:
        if targ is in ['False','false', 'FALSE','partly false', 'Mostly false', 'Partly false', 'PARTLY FALSE','Mostly False', '
       'News', 'Partly False', 'Misleading/False', 'Mixture',
       'no evidence', 'MOSTLY FALSE', 'Misattributed', 'Unproven',
       'Miscaptioned', 'Fake', 'Half True', 'PARTLY TRUE', 'Not true',
       'Scam', "(Org. doesn't apply rating)", 'Partially false',
       'MOSTLY TRUE', 'Partly true', 'mislEADING', 'NO EVIDENCE',
       'half true', 'false and misleading', 'mostly false', nan,
       'HALF TRUE', 'Two Pinocchios', 'Suspicions', 'Partly FALSE',
       'PANTS ON FIRE'
       'Unlikely']

# Encoding the target coulmns to 0 and 1

In [ ]:
# Encode the labels
le = preprocessing.LabelEncoder()
le.fit(df['target'].values)
df['target'] = le.transform(df['target'].values)
df


# Split raw data without cleaning to train and validation

In [ ]:
# Split in train and validation
train_nc_df, validation_nc_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
validation_nc_df.target

# Cleaning the data

In [ ]:
# Custom transformer to implement sentence cleaning
class TextCleanerTransformer(TransformerMixin):
    def __init__(self, tokenizer, stemmer, regex_list,
                 lower=True, remove_punct=True):
        self.tokenizer = tokenizer
        self.stemmer = stemmer
        self.regex_list = regex_list
        self.lower = lower
        self.remove_punct = remove_punct
        
    def transform(self, X, *_):
        X = list(map(self._clean_sentence, X))
        return X
    
    def _clean_sentence(self, sentence):
        
        # Replace given regexes
        for regex in self.regex_list:
            sentence = re.sub(regex[0],regex[1], sentence)
            
        # lowercase
        if self.lower:
            sentence = sentence.lower()

        # Split sentence into list of words
        words = self.tokenizer.tokenize(sentence)
            
        # Remove punctuation
        if self.remove_punct:
            words = list(filter(lambda x: x not in string.punctuation, words))

        # Stem words
        if self.stemmer:
            words = map(self.stemmer.stem, words)

        # Join list elements into string
        sentence = " ".join(words)
        
        return sentence
    
    def fit(self, *_):
        return self

# Cleaning the data by function not class

In [ ]:

tokenizer = WordPunctTokenizer()

def remove_html_tags(text):
    soup = BeautifulSoup(text)
    return soup.get_text()

def remove_punct(text):
    #remove everything except words, digits and space
    text = re.sub(r'[^\w\s]','',text) 
        
    #regex often miss the underscore so let's remove that as well
    text = re.sub(r'\_','',text)
    text = text.lower()
    
    return text

def remove_stopwords(text, stopwords):
    tokens = tokenizer.tokenize(text)
    tokens = [tok.lower() for tok in tokens]
    if stopwords:
        tokens = [tok for tok in tokens if tok not in stopwords]
    tokens = [stemmer.stem(tok) for tok in tokens]

    text_processed = ' '.join(tokens)
    return text_processed



def preprocessing_(df):
    """
    Implement the three above functions in the respective order to remove html tags, punctuations and stopwords
    Hint: Use the apply function.
    
    """
    df_ = df.copy()
    
    #df_['text'] = df_['text'].apply(...).apply(...).apply(...)
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    df_['text'] = df_['text'].apply(remove_html_tags).apply(remove_punct).apply(remove_stopwords,stopwords=en_stopwords)
    return df_

In [ ]:
# Initialize a tokenizer and a stemmer
tokenizer = WordPunctTokenizer()
stemmer = SnowballStemmer("english", ignore_stopwords=True)
#regex_list = [("<[^>]*>", "")]
regex_list = [("\#","")]

#cleaner = TextCleanerTransformer(tokenizer, stemmer, regex_list)
#docs = cleaner.transform(train_df.text.values)

# Define pipline and predict MultinomialNB()

In [ ]:

# Build the pipeline
text_clf = Pipeline([('prep', TextCleanerTransformer(tokenizer, stemmer, regex_list)),
                   ('vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
                   ('tfidf', TfidfTransformer()),
                   ('clf', MultinomialNB())])
# Train the classifier
##text_clf.fit(map(str, train_df['text'].values), train_df['sentiment'].values)
text_clf.fit(map(str, train_nc_df['text'].values), train_nc_df['target'].values)

##predicted = text_clf.predict(map(str, validation_df['text'].values))
##np.mean(predicted == validation_df['sentiment'])

predicted = text_clf.predict(map(str, validation_nc_df['text'].values))
np.mean(predicted == validation_nc_df['target'])

# Define pipeline and predict RandomForrestcalssifier

In [ ]:
# Build the pipeline
text_clf = Pipeline([('prep', TextCleanerTransformer(tokenizer, stemmer, regex_list)),
                   ('vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
                   ('tfidf', TfidfTransformer()),
                   ('classifier', RandomForestClassifier(random_state = 42))])
# Train the classifier
##text_clf.fit(map(str, train_df['text'].values), train_df['sentiment'].values)
text_clf.fit(map(str, train_nc_df['text'].values), train_nc_df['target'].values)

##predicted = text_clf.predict(map(str, validation_df['text'].values))
##np.mean(predicted == validation_df['sentiment'])

predicted = text_clf.predict(map(str, validation_nc_df['text'].values))
np.mean(predicted == validation_nc_df['target'])

# Check classification result

In [ ]:
# check the results
##print(classification_report(y_dev, y_dev_pred))
print(classification_report(validation_nc_df['target'].values,predicted))

# How to counts the number of occurrences of different tokens in a list and whole corpus

def build_vocabulary():
    vocabulary = Counter()

    for doc in docs:
        words = doc.split()
        vocabulary.update(words)
    
    return OrderedDict(vocabulary.most_common())
# turn into a list of tuples and get the first 20 items
list(build_vocabulary().items())[:20]

# Information Extraction from text

# Here we are disabling the synctatic parser from pipeline to improve speed.
nlp = spacy.load('en_core_web_sm', disable=['parser'])
en_stopwords = nlp.Defaults.stop_words
###################################################Clean data with the functions not classes
df_clean = preprocessing_(df)
df_clean

# define docs as a list of sentences

docs = list(df_clean.text.values)

#We are going to use the function pipe to process all documents.
# One of the strenghts for SpaCy is the parallel processing using all your computer cores.
# In this step, SpaCy performs the NLP pipeline for all the docs, so it may take a while

#docs = list(nlp.pipe(docs))

In [ ]:
#docs[0].ents

#Matcher ans pass vocabulary to it
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab) # Pass the vocabulary object to Matcher.__init__()
pattern = [{'POS': 'ADV'},{'POS':'ADJ'}]
matcher.add('word',None,pattern)

##Extracting adjectives,
for i, doc in enumerate(docs[:400]):
    #print(i,doc)
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]  # the matched span
        print(i, start, end, span)
   

# Extracting locations

for i, doc in enumerate(docs[:400]):
    for e in doc.ents:
        if e.label_ == 'GPE':   
            print(i, e.text, e.start_char, e.end_char)

# Extracting entities

matcher = Matcher(nlp.vocab)
#
pattern = [{'ENT_TYPE':'GPE'}]
matcher.add('LOC', None, pattern)
#matcher.add(...)
most_common_ents_list = []
for doc in docs:
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]  # the matched span
        span_text = span.text  # the span as a string
        most_common_ents_list.append(span_text)
most_common_ents = Counter(most_common_ents_list).most_common(10)
most_common_ents

# Adding even more features

In [ ]:
#number of adjectives
name_of__adjs = [[token for token in doc if token.pos_ == 'ADJ'] for doc in docs]
n_adjs = [len(internal_list) for internal_list in name_of__adjs]
#####df['n_adjs'] = n_adjs
##Emoji

nlp = English()  # We only want the tokenizer, so no need to load a model
matcher = Matcher(nlp.vocab)

pos_patterns = [{'TEXT':{"REGEX": r"\:\)"}}] #- For Positive emoji let's use ":)"
neg_patterns = [{'TEXT':{"REGEX": r"\:\("}}] #- For Negative emoji let's use ":("

# Hint - Don't forget to escape the special character "(" and ")"
# YOUR CODE HERE
#raise NotImplementedError()

def count_emoji_matches(pattern, docs = docs):
    matcher = Matcher(nlp.vocab)
    matcher.add("EMOJIS", None, pattern)
    
    n_emojis = []
    for doc in docs:
        matches = matcher(doc)
        emojis_count = len(matches)
        for match in matches:
            emojis_count += 1
        n_emojis.append(emojis_count)
            
    return n_emojis

positive_emojis_count = sum(count_emoji_matches(pos_patterns))
negative_emojis_count = sum(count_emoji_matches(neg_patterns))



##df['positive_emojis_count'] = count_emoji_matches(pos_patterns)
##df['negative_emojis_count'] = count_emoji_matches(neg_patterns)

# Adding more extra features

In [ ]:
df_clean['length'] = df_clean['text'].map(len)# we test if this feature is ok or not by plotting in the next cell we can do it for all the new features if it is skew to right and left it is important otherwise not inmportant
df_clean['words'] = df_clean['text'].str.split().map(len)
stop_words = nltk.corpus.stopwords.words('english')
df_clean['words_not_stopword'] = df_clean['text'].apply(lambda x: len([t for t in x.split() if t not in stop_words]))
df_clean['commas'] = df_clean['text'].str.count(',')
df_clean['upper'] = df_clean['text'].map(lambda x: map(str.isupper, x)).map(sum)
df_clean['capitalized'] = df_clean['text'].map(lambda x: map(str.istitle, x)).map(sum)
#get the average word length
#df_clean['avg_word_length'] = df_clean['text'].apply(lambda x: np.mean([len(t) for t in x.split() if t not in stop_words]) if len([len(t) for t in x.split(' ') if t not in stop_words]) > 0 else 0)
#number of adjective
df_clean['n_adjs'] = n_adjs
df_clean['positive_emojis_count'] = count_emoji_matches(pos_patterns)
df_clean['negative_emojis_count'] = count_emoji_matches(neg_patterns)
df_clean


In [ ]:
df_clean.isnull().sum()

# Is this feature usefull?(by the plot the length is  useful)

In [ ]:
ax_list = df_clean.hist(column='length', by='target', bins=50,figsize=(12,4))
ax_list[0].set_xlim((0,300))
ax_list[1].set_xlim((0,300))

# Class for choosing extra features

In [ ]:
class Selector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a column from the dataframe to perform additional transformations on
    """ 
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        return self
    

class TextSelector(Selector):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def transform(self, X):
        return X[self.key]
    
    
class NumberSelector(Selector):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def transform(self, X):
        return X[[self.key]]
    
    

# Define a pipeline

In [ ]:
text = Pipeline([
                ('selector', TextSelector("text")),
                ('tfidf', TfidfVectorizer())
            ])

length =  Pipeline([
                ('selector', NumberSelector("length")),
                ('standard', StandardScaler())
            ])

words =  Pipeline([
                ('selector', NumberSelector(key='words')),
                ('standard', StandardScaler())
            ])
words_not_stopword =  Pipeline([
                ('selector', NumberSelector(key='words_not_stopword')),
                ('standard', StandardScaler())
            ])
#avg_word_length =  Pipeline([
 #               ('selector', NumberSelector(key='avg_word_length')),
  #              ('standard', StandardScaler())
   #         ])
commas =  Pipeline([
                ('selector', NumberSelector(key='commas')),
                ('standard', StandardScaler()),
            ])
upper =  Pipeline([
                ('selector', NumberSelector(key='upper')),
                ('standard', StandardScaler()),
            ])
capitalized =  Pipeline([
                ('selector', NumberSelector(key='capitalized')),
                ('standard', StandardScaler()),
            ])

#n_adjs = Pipeline(['selector', NumberSelector(key='n_adjs'),
                  # ('standard', StandardScaler()),
                 # ])

positive_emojis_count  = Pipeline([
                ('selector', NumberSelector("positive_emojis_count"))
            ])
negative_emojis_count = Pipeline([
               ('selector', NumberSelector("negative_emojis_count"))
            ])


feats = FeatureUnion([('text', text), 
                      ('length', length),
                      ('words', words),
                      ('words_not_stopword', words_not_stopword),
                      #('avg_word_length', avg_word_length),
                      ('commas', commas),
                     ('upper', upper),
                     ('capitalized', capitalized),
                     #('n_adjs',n_adjs),
                      ('positive_emojis_count',positive_emojis_count),
                      ('negative_emojis_count',negative_emojis_count),
                     ])

feature_processing = Pipeline([('feats', feats)])

In [ ]:
#df_clean = df_clean.reset_index(drop = True)
#df_clean

# Split train and test of clean data and fitting the model and predicting

In [ ]:
# Split in train and validation
train_cl_data, test_cl_data = train_test_split(df_clean, test_size=0.2, random_state=42)



In [ ]:
train_cl_data.isnull().sum()

In [ ]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
    #('classifier', LinearRegression()),
])

pipeline.fit(train_cl_data, train_cl_data.target)

preds = pipeline.predict(test_cl_data)
np.mean(preds == test_cl_data.target)

# How to reduce the features and choose the more important one
1- Feature Selection from chi-squared
2-Feature selection with SVD and PCA

# # The following will plot the  most dependent features from the chi-squared values.

In [ ]:
nltk.download("stopwords") 
stopword_list =list(nltk.corpus.stopwords.words("english"))

In [ ]:
#preprocess to run the following cell about chi_values
#df is already clean data
#Encode the lables y_train and y_dev
vec = TfidfVectorizer(ngram_range=(1,2))
X_train_vec = vec.fit_transform(train_cl_data.text)
X_test_vec = vec.transform(test_cl_data.text)


In [ ]:
chi_values, p_values = chi2(X_train_vec, train_cl_data.target.values)
print(chi_values, p_values)

In [ ]:
feature_names = vec.get_feature_names()

cla()   # Clear axis
close() # Close a figure window

figure(figsize=(12,10))
zipped_chi_squared = zip(feature_names, chi_values)
sorted_chi_values = sorted(zipped_chi_squared, key=lambda x:x[1]) 
top_chi_values = list(zip(*sorted_chi_values[-30:]))

x = range(len(top_chi_values[1]))
labels = top_chi_values[0]
barh(x, list(top_chi_values)[1], align='center', alpha=.2, color='g')
yticks(x, labels)
xlabel('$\chi^2$')
show()

# Ch2 choose best features

In [ ]:
ch2 = SelectKBest(chi2, k=10)
ch2.fit(X_train_vec, train_cl_data.target)

most_important_features = [feature_names[i] for i in ch2.get_support(indices=True)]
for f in most_important_features:
    print(f)

# Prediction

In [ ]:
X_train_chi = ch2.transform(X_train_vec)
X_test_chi = ch2.transform(X_test_vec)

clf = MultinomialNB()
%timeit clf.fit(X_train_chi, train_cl_data.target)

y_pred = clf.predict(X_test_chi)